In [1]:
!pip install nltk

In [130]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

In [4]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
len(word_tokenize(document.lower()))

1018

In [36]:
def tokenize_document_build_vocab(document):
    document=document.lower().split('\n')
    tokenized_document=[]
    vocab={'<UNK>':0}
    for sentence in document:
      tokenized_sentence=word_tokenize(sentence)
      tokenized_document.append(tokenized_sentence)
      for word in tokenized_sentence:
        if word not in vocab:
          vocab[word]=len(vocab)
    return tokenized_document,vocab

In [40]:
def document_to_index(tokenize_document,vocab):
  indexed_document=[]
  for sentence in tokenize_document:
    indexed_sentence=[]
    for word in sentence:
      indexed_sentence.append(vocab[word])
    indexed_document.append(indexed_sentence)
  return indexed_document

In [43]:
tokenized_doc,vocab=tokenize_document_build_vocab(document)
indexed_doc=document_to_index(tokenized_doc,vocab)

In [49]:
data=[]
for sentence in indexed_doc:
  i=0
  while(i+2<=len(sentence)):
    data.append(sentence[:i+2])
    i+=1

942

In [50]:
max=0
for dataPoint in data:
  if len(dataPoint)>max:
    max=len(dataPoint)

62

In [53]:
for i,dataPoint in enumerate(data):
  data[i]=[0]*(max-len(dataPoint))+dataPoint

942

In [84]:
data=torch.tensor(data,dtype=torch.long)
X=data[:,:-1]
Y=data[:,-1]

tensor(2)

In [87]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X=X
    self.Y=Y

  def __len__(self):
    return len(self.X)

  def __getitem__(self,index):
    return self.X[index],self.Y[index]


In [88]:
myDataSet=dataset(X,Y)

In [92]:
dataloader=DataLoader(myDataSet,batch_size=32,shuffle=True)

In [125]:
class LSTM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,100)
    self.lstm=nn.LSTM(100,128,batch_first=True)
    self.fc=nn.Linear(128,vocab_size)
  def forward(self,X):
    out=self.embedding(X)
    _,(out,_)=self.lstm(out)
    out=self.fc(out.squeeze_(0))
    return out

In [126]:
model=LSTM(len(vocab))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [131]:
epoches=50
learning_rate=0.001
criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [134]:
#training loop
for epoch in range(epoches):
  total_loss=0
  for x,y in dataloader:
    x,y=x.to(device),y.to(device)
    optimizer.zero_grad()
    y_pred=model(x)
    loss=criterion(y_pred,y)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  print(f"epoch: {epoch+1}, loss: {(total_loss/32):.4f}")


epoch: 1, loss: 5.2191
epoch: 2, loss: 4.6356
epoch: 3, loss: 4.2620
epoch: 4, loss: 3.9154
epoch: 5, loss: 3.5975
epoch: 6, loss: 3.2650
epoch: 7, loss: 2.9634
epoch: 8, loss: 2.6774
epoch: 9, loss: 2.4174
epoch: 10, loss: 2.1803
epoch: 11, loss: 1.9545
epoch: 12, loss: 1.7620
epoch: 13, loss: 1.5926
epoch: 14, loss: 1.4107
epoch: 15, loss: 1.2671
epoch: 16, loss: 1.1299
epoch: 17, loss: 1.0088
epoch: 18, loss: 0.9011
epoch: 19, loss: 0.8102
epoch: 20, loss: 0.7272
epoch: 21, loss: 0.6501
epoch: 22, loss: 0.5946
epoch: 23, loss: 0.5405
epoch: 24, loss: 0.4909
epoch: 25, loss: 0.4531
epoch: 26, loss: 0.4135
epoch: 27, loss: 0.3910
epoch: 28, loss: 0.3604
epoch: 29, loss: 0.3371
epoch: 30, loss: 0.3143
epoch: 31, loss: 0.2927
epoch: 32, loss: 0.2763
epoch: 33, loss: 0.2616
epoch: 34, loss: 0.2494
epoch: 35, loss: 0.2384
epoch: 36, loss: 0.2306
epoch: 37, loss: 0.2188
epoch: 38, loss: 0.2119
epoch: 39, loss: 0.2049
epoch: 40, loss: 0.1951
epoch: 41, loss: 0.1875
epoch: 42, loss: 0.1836
e

In [135]:
model.eval()

LSTM(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=289, bias=True)
)

In [162]:
def predict(model,ogtext,vocab):
  text_index=[]
  text=word_tokenize(ogtext.lower())
  for t in text:
    if vocab.get(t)!=None:
      text_index.append(vocab.get(t))
    else:
      text_index.append(vocab.get('<UNK>'))
  text_index=torch.tensor([0]*(max-len(text_index))+text_index,dtype=torch.long)
  with torch.no_grad():
    y_pred=model(torch.tensor(text_index).unsqueeze(0).to(device))
  predict = torch.argmax(y_pred).item()
  return  ogtext + " " + list(vocab.keys())[predict]

In [172]:
predict(model,"Yes, once you make the payment you will be able to see all the past content in your",vocab)

'Yes, once you make the payment you will be able to see all the past content in your dashboard'